In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))



Author: Prof. Manoel Gadi

Contact: manoelgadi@gmail.com

Teaching Web: http://mfalonso.pythonanywhere.com

Linkedin: https://www.linkedin.com/in/manoel-gadi-97821213/

Github: https://github.com/manoelgadi

Last revision: 27/October/2022



# Session 7 - Introduction to basic statistics in Python with Scipy

# 1. Introduction: ```scipy.stats``` module

The SciPy library has many modules, each aiming to solve or handle specific purposes related to science. The scope of the modules themselves can be daunting, and without knowing when and where to use them, it is easy to overlook the library's usefulness. This exercise sticks to using one of the basic module called ```scipy.stats```. 

```scipy.stats``` contains a large number of probabilitiy distributions and statistical functions. It is an extension to NumPy's own stats module, though the number and variety of functions are much more diverse.

The SciPy library has its own [documentation](https://docs.scipy.org/doc/scipy/reference/), which you could refer to.

# 2. Loading libraries and dataset

You can import specific modules of SciPy using ```import scipy.[module_name]``` format. Note that it is convention to import specific module directly to the workspace for SciPy. This avoids possible clash of named global variables that are defined in other modules. 

Also note that SciPy is built on top of NumPy, NumPy is automatically imported too when SciPy is imported. However, calling NumPy separately avoids any confusion when calling a module specific function.

In [1]:
# import libraries numpy, pandas, and scipy.stats module

import numpy as np
import pandas as pd
import scipy.stats as stats

In [2]:
import sqlite3
conn = sqlite3.connect("db\company_balancesheet_database.db")


In [3]:
# import dataset using pd.read_sql() function
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
df = pd.read_sql("""SELECT * FROM balance_sheet;""", conn)

In [5]:
# view first few rows of the dataset using pandas dataframe.head() function
df.head()

,id,nif_fical_number_id,company_name,CNAE,p10000_TotalAssets_h0,p10000_TotalAssets_h1,p10000_TotalAssets_h2,p20000_OwnCapital_h0,p20000_OwnCapital_h1,p20000_OwnCapital_h2,...,p40100_40500_SalesTurnover_h0,p40100_40500_SalesTurnover_h1,p40100_40500_SalesTurnover_h2,p40800_Amortization_h0,p40800_Amortization_h1,p40800_Amortization_h2,p49100_Profit_h0,p49100_Profit_h1,p49100_Profit_h2,detailed_status
0,1,A28015865,"Telefonica, SA",6420,115066000.000,123641000.000,120329000.000,26618000.000,28385000.000,25436000.000,...,52455000.000,52574000.000,55665000.000,-9396000.000,-9649000.000,-9704000.000,6791000.000,5469000.000,3525000.000,Activa
1,2,A39000013,Banco Santander SA,6419,1444305000.000,1339125000.000,1340260000.000,106833000.000,102699000.000,98753000.000,...,50321000.000,46192000.000,48130000.000,0.000,0.000,0.000,0.000,0.000,0.000,0
2,3,A78374725,Repsol SA.,7010,59857000.000,64849000.000,63077000.000,30067000.000,31115000.000,28696000.000,...,42378000.000,35679000.000,42265000.000,-2399000.000,-2529000.000,-2988000.000,2789000.000,1911000.000,-2440000.000,Activa
3,4,A32001166,Peugeot España SA (Extinguida),4511,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,Extinción: La empresa se encuentra en situació...
4,5,A28004885,"Acs, Actividades De Construccion Y Servicios, SA",4299,31880684.000,33373266.000,35279828.000,5168036.000,4985911.000,5256045.000,...,35218839.000,32436917.000,35345782.000,-611218.000,-513934.000,-788001.000,1329227.000,1237430.000,1221298.000,Activa


In [6]:
df['alive'] = df['detailed_status'].apply(lambda x: 1 if x in ('Activa','') else 0)

In [7]:
df['industry'] = df.CNAE.apply(lambda x: int(str(x)[:1])) # industry the first digit of CNAE

In [8]:
df['industry'].value_counts()

4    22419
2     6222
6     5360
1     4989
7     3083
5     3055
8     2099
3     2015
9      754
0        4
Name: industry, dtype: int64

Alive:

* alive: = 1: Company still active at the end of the period
* alive: = 0: Company not in operation at the end of the period

We can use various statistical tools in the scipy.stats module to perform descriptive statistics and statistical tests.

In [9]:
df.columns[4:-2]

Index(['p10000_TotalAssets_h0', 'p10000_TotalAssets_h1',
       'p10000_TotalAssets_h2', 'p20000_OwnCapital_h0', 'p20000_OwnCapital_h1',
       'p20000_OwnCapital_h2', 'p31200_ShortTermDebt_h0',
       'p31200_ShortTermDebt_h1', 'p31200_ShortTermDebt_h2',
       'p32300_LongTermDebt_h0', 'p32300_LongTermDebt_h1',
       'p32300_LongTermDebt_h2', 'p40100_40500_SalesTurnover_h0',
       'p40100_40500_SalesTurnover_h1', 'p40100_40500_SalesTurnover_h2',
       'p40800_Amortization_h0', 'p40800_Amortization_h1',
       'p40800_Amortization_h2', 'p49100_Profit_h0', 'p49100_Profit_h1',
       'p49100_Profit_h2', 'detailed_status'],
      dtype='object')

# 3. Statistical tests

## 3-1. Are the weights normally distributed? (Normality tests)

There are various ways in which we can test if a given set of data is normally distributed or not.

Here we introduce these:
* [```scipy.stats.normaltest(x)```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html#r676), where x is the data array
* [```scipy.stats.shapiro(x)```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html), where x is the data array

Both tests are used to determine whether or not a dataset comes from a normal distribution. For both tests, the hypothesis is as follows:

* H0: Sample comes from a normal distribution
* H1: Sample does not come from a normal distribution.

As in the case of the modules, you can access functions of stats module using ```stats.[function_name]``` format.

In [10]:
# use the normaltest function to find the test statistic and p-value
# of each column, and print each of them.
# since pandas dataframe is also built on top of arrays,
# you can use the dataframe column selection method directly to use the normaltest
for col in df.columns[4:-3]:
    print('Column: {}: {} '.format(col,stats.normaltest(df[col])))

Column: p10000_TotalAssets_h0: NormaltestResult(statistic=204300.39683076745, pvalue=0.0) 
Column: p10000_TotalAssets_h1: NormaltestResult(statistic=197386.6186145027, pvalue=0.0) 
Column: p10000_TotalAssets_h2: NormaltestResult(statistic=197976.26054356323, pvalue=0.0) 
Column: p20000_OwnCapital_h0: NormaltestResult(statistic=175365.02557736172, pvalue=0.0) 
Column: p20000_OwnCapital_h1: NormaltestResult(statistic=169457.63075443846, pvalue=0.0) 
Column: p20000_OwnCapital_h2: NormaltestResult(statistic=167755.74705677677, pvalue=0.0) 
Column: p31200_ShortTermDebt_h0: NormaltestResult(statistic=192068.64334298606, pvalue=0.0) 
Column: p31200_ShortTermDebt_h1: NormaltestResult(statistic=192480.98869902236, pvalue=0.0) 
Column: p31200_ShortTermDebt_h2: NormaltestResult(statistic=193504.2882903775, pvalue=0.0) 
Column: p32300_LongTermDebt_h0: NormaltestResult(statistic=207030.71480894682, pvalue=0.0) 
Column: p32300_LongTermDebt_h1: NormaltestResult(statistic=197810.0499268255, pvalue=0.0

As you can see, the function returns 2 values: the test statistic, and the p-value. If you want to access a particular value of the two, you can use the slicing method or assign both values to a new variable at the same time.

```
# slicing:
p_value = stats.normaltest(x)[1]

# variable assignment
test_statistic, p_value = stats.normaltest(x)
```

In [11]:
print('Column: {}: {} '.format(col,stats.normaltest(df[col])))
print(stats.normaltest(df[col])[0]) # statistic
print(stats.normaltest(df[col])[1]) # p-value

Column: p49100_Profit_h2: NormaltestResult(statistic=132013.88468899758, pvalue=0.0) 
132013.88468899758
0.0


Just to verify the normality, use the [```stats.shapiro(x)```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html) function to conduct the shapiro test on the columns.

In [12]:
# use the shapiro function to find the test statistic and p-value
# of each column, and print each of them.
# shapiro test also returns test-statistic and p-value respectively

for col in df.columns[4:-3]:
    print('Column: {}: {} '.format(col,stats.shapiro(df[col])))


Column: p10000_TotalAssets_h0: ShapiroResult(statistic=0.006810009479522705, pvalue=0.0) 
Column: p10000_TotalAssets_h1: ShapiroResult(statistic=0.008111357688903809, pvalue=0.0) 
Column: p10000_TotalAssets_h2: ShapiroResult(statistic=0.008340120315551758, pvalue=0.0) 
Column: p20000_OwnCapital_h0: ShapiroResult(statistic=0.025196373462677002, pvalue=0.0) 
Column: p20000_OwnCapital_h1: ShapiroResult(statistic=0.027913928031921387, pvalue=0.0) 
Column: p20000_OwnCapital_h2: ShapiroResult(statistic=0.02779608964920044, pvalue=0.0) 
Column: p31200_ShortTermDebt_h0: ShapiroResult(statistic=0.0124778151512146, pvalue=0.0) 
Column: p31200_ShortTermDebt_h1: ShapiroResult(statistic=0.014345288276672363, pvalue=0.0) 
Column: p31200_ShortTermDebt_h2: ShapiroResult(statistic=0.013727664947509766, pvalue=0.0) 
Column: p32300_LongTermDebt_h0: ShapiroResult(statistic=0.009758532047271729, pvalue=0.0) 
Column: p32300_LongTermDebt_h1: ShapiroResult(statistic=0.009930074214935303, pvalue=0.0) 
Column: 

/home/manuel/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:1761: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


```
# slicing:
p_value = stats.shapiro(x)[1]

# variable assignment
shapiro_statistic, p_value = stats.shapiro(x)
```

In [13]:
print('Column: {}: {} '.format(col,stats.shapiro(df[col])))
print(stats.shapiro(df[col])[0]) # statistic
print(stats.shapiro(df[col])[1]) # p-value

Column: p49100_Profit_h2: ShapiroResult(statistic=0.04196447134017944, pvalue=0.0) 
0.04196447134017944
0.0


## 3-2. Skewness and Kurtosis

The normaltest uses kurtosis and skewness to test normality. We can check the actual kurtosis and skewness values by using [```stats.kurtosis```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosis.html) and [```stats.skew```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skew.html) functions (note: kurtosis returns excess kurtosis value). You can also run a separate tests using [```stats.kurtosistest```](https://docs.scipy.org/doc/scipy-0.16.0/reference/generated/scipy.stats.kurtosistest.html) and [```stats.skewtest```](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.skewtest.html). 

In [14]:
# find the excess kurtosis and skewness values
# of each column, and the print the values
# using stats.kurtosis(x) and stats.skew(x)

for col in df.columns[4:-3]:
    print('Column:{} / kurtosis={} / skweness={}'.format(col,stats.kurtosis(df[col]),stats.skew(df[col])))


Column:p10000_TotalAssets_h0 / kurtosis=15003.432286983887 / skweness=109.49533461800223
Column:p10000_TotalAssets_h1 / kurtosis=12387.580230351337 / skweness=98.16277290608836
Column:p10000_TotalAssets_h2 / kurtosis=12720.641233540084 / skweness=99.07282309540646
Column:p20000_OwnCapital_h0 / kurtosis=6699.479951320811 / skweness=68.34931942253174
Column:p20000_OwnCapital_h1 / kurtosis=5594.468913308029 / skweness=61.79730529136646
Column:p20000_OwnCapital_h2 / kurtosis=5277.07440670696 / skweness=60.01815182197934
Column:p31200_ShortTermDebt_h0 / kurtosis=9899.212448112425 / skweness=90.25026624996731
Column:p31200_ShortTermDebt_h1 / kurtosis=10207.245996912494 / skweness=90.82210715192645
Column:p31200_ShortTermDebt_h2 / kurtosis=10608.880908110608 / skweness=92.31644470769314
Column:p32300_LongTermDebt_h0 / kurtosis=15118.240826864598 / skweness=114.37816406149234
Column:p32300_LongTermDebt_h1 / kurtosis=11402.884927673811 / skweness=98.98370090997265
Column:p32300_LongTermDebt_h2 

```
# getting the value:
kurtosis_statistic = stats.kurtosis(x)

# variable assignment
skewness_statistic = stats.skew(x)
```

In [15]:
kurtosis_statistic = stats.kurtosis(df[col])
skewness_statistic = stats.skew(df[col])
print('Column: {}: skewness={} '.format(col,skewness_statistic)) # statistic
print('Column: {}: kurtosis={} '.format(col,kurtosis_statistic)) # p-value


Column: p49100_Profit_h2: skewness=30.91493286489067 
Column: p49100_Profit_h2: kurtosis=2125.8930718038105 


## 3-3. Have people's weight changed in general before and after the treatments? (one-sample t-test)

Since we already have the values for difference in weight pre and post treatment, we can use one-sample paired t-test on the 'difwt' column to see whether there is a significant change in weights, where the hypothesis is:

* H0: Mean difference is equal to 0
* H1: Mean difference is not equal to 0

We can use the [```stats.ttest_1samp(x, mean)```](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ttest_1samp.html) to conduct the one-sample t-test, where x is the data and mean is the population mean you want to test the data against (in this case, 0). The function returns the test statistic and the p-value respectively.

In [16]:
# Use the stats.ttest_1samp function to see whether the difference 
# of last year and this year for a given variable is different from population mean 0
# and print the result
print('p10000_TotalAssets:',stats.ttest_1samp(df['p10000_TotalAssets_h0']-df['p10000_TotalAssets_h1'], 0))
print('p20000_OwnCapital:',stats.ttest_1samp(df['p20000_OwnCapital_h0']-df['p20000_OwnCapital_h1'], 0))
print('p31200_ShortTermDebt:', stats.ttest_1samp(df['p31200_ShortTermDebt_h0']-df['p31200_ShortTermDebt_h1'], 0))
print('p32300_LongTermDebt:', stats.ttest_1samp(df['p32300_LongTermDebt_h0']-df['p32300_LongTermDebt_h1'], 0))
print('p40100_40500_SalesTurnover:', stats.ttest_1samp(df['p40100_40500_SalesTurnover_h0']-df['p40100_40500_SalesTurnover_h1'], 0))
print('p40800_Amortization:', stats.ttest_1samp(df['p40800_Amortization_h0']-df['p40800_Amortization_h1'], 0))
print('p49100_Profit:', stats.ttest_1samp(df['p49100_Profit_h0']-df['p49100_Profit_h1'], 0))


p10000_TotalAssets: Ttest_1sampResult(statistic=-1.7665518221879424, pvalue=0.07730941599304499)
p20000_OwnCapital: Ttest_1sampResult(statistic=-2.172171889415615, pvalue=0.029847383082839296)
p31200_ShortTermDebt: Ttest_1sampResult(statistic=-4.366143301735749, pvalue=1.2671317346758155e-05)
p32300_LongTermDebt: Ttest_1sampResult(statistic=-2.307657401586613, pvalue=0.021022261819747562)
p40100_40500_SalesTurnover: Ttest_1sampResult(statistic=-2.035018760050049, pvalue=0.041854249101523425)
p40800_Amortization: Ttest_1sampResult(statistic=2.2243029216430057, pvalue=0.026132517794939032)
p49100_Profit: Ttest_1sampResult(statistic=0.10295542723365515, pvalue=0.9179987597078809)


```
# slicing:
p_value = stats.ttest_1samp(x)[1]

# variable assignment
shapiro_statistic, p_value = stats.ttest_1samp(x)
```

In [17]:
print('Column: {}: {} '.format('p10000_TotalAssets',stats.ttest_1samp(df['p10000_TotalAssets_h0']-df['p10000_TotalAssets_h1'], 0)))
print(stats.ttest_1samp(df['p10000_TotalAssets_h0']-df['p10000_TotalAssets_h1'], 0)[0]) # statistic
print(stats.ttest_1samp(df['p10000_TotalAssets_h0']-df['p10000_TotalAssets_h1'], 0)[1]) # p-value

Column: p10000_TotalAssets: Ttest_1sampResult(statistic=-1.7665518221879424, pvalue=0.07730941599304499) 
-1.7665518221879424
0.07730941599304499


## 3-4. Have the value not changed overtime? (paired t-test)

Now we can check if the actual differences have changed using a paired t-test, as each pre-weight and post-weight has to match for each individual. The hypothesis is similar:

* H0: Mean(p10000_TotalAssets_h0) = Mean(p10000_TotalAssets_h1)
* H1: Mean(p10000_TotalAssets_h0) != Mean(p10000_TotalAssets_h1).

We can use the [```stats.ttest_rel(a,b)```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) to conduct a paired t-test, where a and b are each of the samples. The function returns the test-statistic and the p-value respectively.

In [18]:
# use the stats.ttest_rel function to see whether the values of 'prewt' and 'postwt'
# for each individual are significantly different
# and print the result
print('p10000_TotalAssets:',stats.ttest_rel(df['p10000_TotalAssets_h0'],df['p10000_TotalAssets_h1']))
print('p20000_OwnCapital:',stats.ttest_rel(df['p20000_OwnCapital_h0'],df['p20000_OwnCapital_h1']))
print('p31200_ShortTermDebt:', stats.ttest_rel(df['p31200_ShortTermDebt_h0'],df['p31200_ShortTermDebt_h1']))
print('p32300_LongTermDebt:', stats.ttest_rel(df['p32300_LongTermDebt_h0'],df['p32300_LongTermDebt_h1']))
print('p40100_40500_SalesTurnover:', stats.ttest_rel(df['p40100_40500_SalesTurnover_h0'],df['p40100_40500_SalesTurnover_h1']))
print('p40800_Amortization:', stats.ttest_rel(df['p40800_Amortization_h0'],df['p40800_Amortization_h1']))
print('p49100_Profit:', stats.ttest_rel(df['p49100_Profit_h0'],df['p49100_Profit_h1']))

p10000_TotalAssets: Ttest_relResult(statistic=-1.7665518221879424, pvalue=0.07730941599304499)
p20000_OwnCapital: Ttest_relResult(statistic=-2.172171889415615, pvalue=0.029847383082839296)
p31200_ShortTermDebt: Ttest_relResult(statistic=-4.366143301735749, pvalue=1.2671317346758155e-05)
p32300_LongTermDebt: Ttest_relResult(statistic=-2.307657401586613, pvalue=0.021022261819747562)
p40100_40500_SalesTurnover: Ttest_relResult(statistic=-2.035018760050049, pvalue=0.041854249101523425)
p40800_Amortization: Ttest_relResult(statistic=2.2243029216430057, pvalue=0.026132517794939032)
p49100_Profit: Ttest_relResult(statistic=0.10295542723365515, pvalue=0.9179987597078809)


```
# slicing:
p_value = stats.ttest_rel(x)[1]

# variable assignment
shapiro_statistic, p_value = stats.ttest_rel(x)
```

In [19]:
print('p10000_TotalAssets:',stats.ttest_rel(df['p10000_TotalAssets_h0'],df['p10000_TotalAssets_h1']))
print(stats.ttest_rel(df['p10000_TotalAssets_h0'],df['p10000_TotalAssets_h1'])[0]) # statistic
print(stats.ttest_rel(df['p10000_TotalAssets_h0'],df['p10000_TotalAssets_h1'])[1]) # p-value

p10000_TotalAssets: Ttest_relResult(statistic=-1.7665518221879424, pvalue=0.07730941599304499)
-1.7665518221879424
0.07730941599304499


## 3-5. Are the effect of each treatment different from one another? (one-way ANOVA)

Since there are 3 groups, we need to use the ANOVA test to determine whether or not the means (difference in weight) are the same. The hypothesis is as follows:

* H0: Mean(group1) = Mean(group2) = Mean(group3)
* H1: at least one pair has different means

We can use the [```stats.f_oneway(a,b,c...)```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html) to conduct a one-way anova test, where a, b, c.. are each of the samples. The function returns the test statistic and the p-value respectively.

In [20]:
# use the stats.f_oneway function to see whether the means of 'difwt' column
# are significantly different for at least one pair
print(stats.f_oneway(df[df['industry'] == 0]['p10000_TotalAssets_h0'], 
                     df[df['industry'] == 1]['p10000_TotalAssets_h0']
                    ))

print(stats.f_oneway(df[df['industry'] == 0]['p10000_TotalAssets_h0'], 
                     df[df['industry'] == 1]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 2]['p10000_TotalAssets_h0']
                    ))

print(stats.f_oneway(df[df['industry'] == 0]['p10000_TotalAssets_h0'], 
                     df[df['industry'] == 1]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 2]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 3]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 4]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 5]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 6]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 7]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 8]['p10000_TotalAssets_h0'],
                     df[df['industry'] == 9]['p10000_TotalAssets_h0'],
                    ))

# you can carry out additional tukey's test using
# the statsmodels library :)

F_onewayResult(statistic=0.03286933991611936, pvalue=0.8561402256292817)
F_onewayResult(statistic=0.8536202215287285, pvalue=0.42589806807913066)
F_onewayResult(statistic=18.29812067235471, pvalue=8.934723780227242e-31)


```
# slicing:
p_value = stats.f_oneway(x,y,z)[1]

# variable assignment
shapiro_statistic, p_value = stats.f_oneway(x,y,z)
```

In [21]:
print(stats.f_oneway(df[df['industry'] == 0]['p10000_TotalAssets_h0'], 
                     df[df['industry'] == 1]['p10000_TotalAssets_h0']
                    ))

print(stats.f_oneway(df[df['industry'] == 0]['p10000_TotalAssets_h0'], 
                     df[df['industry'] == 1]['p10000_TotalAssets_h0']
                    )[0]) # statistic
print(stats.f_oneway(df[df['industry'] == 0]['p10000_TotalAssets_h0'], 
                     df[df['industry'] == 1]['p10000_TotalAssets_h0']
                    )[1]) # p-value

F_onewayResult(statistic=0.03286933991611936, pvalue=0.8561402256292817)
0.03286933991611936
0.8561402256292817


---